# Image Classifier for Dogs and Cats

## Features:
    1. Data augmentation

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

plt.rcParams['figure.figsize'] = 15, 8

In [2]:
from keras import Input, Model
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from keras import optimizers, losses, metrics
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

Using TensorFlow backend.


In [15]:
class CatsAndDogs:

    def __init__(self, loss='binary', metric='binary',
                 optimizer='adam', lr=1e-4, epochs=20, image_size=150,
                 batch_size=20, augmentation=True, plotting=True,
                 dropout=False, regularizer=False):

        self.loss = loss
        self.metric = metric
        self.optimizer = optimizer
        self.lr = lr
        self.epochs = epochs
        self.image_size = image_size
        self.batch_size = batch_size
        self.augmentation = augmentation
        self.plotting = plotting
        self.dropout = dropout
        self.regularizer = regularizer
        self.DATADIR = None
        self.LOGDIR = None

    def _load_data(self):

        self.DATADIR = '/Volumes/Study/GitHub/data/cats-dogs/'
        self.LOGDIR = '/Volumes/Study/GitHub/neural-nets/graphs/cats-dogs/'

        if not os.path.exists(self.LOGDIR):
            os.mkdir(self.LOGDIR, 755)
            
        print('Data directory',self.DATADIR)
        print('Log directory:',self.LOGDIR)
        
        self.train_dir = os.path.join(self.DATADIR + 'training/')
        self.validation_dir = os.path.join(self.DATADIR + 'validation/')
        self.test_dir = os.path.join(self.DATADIR + 'test/')

    def data_generator(self):
        if self.augmentation:
            train_datagen = ImageDataGenerator(rescale=1.0/255,
                                               rotation_range=40,
                                               width_shift_range=0.2,
                                               height_shift_range=0.2,
                                               shear_range=0.2,
                                               zoom_range=0.2,
                                               horizontal_flip=True,
                                               fill_mode='nearest')

            validation_datagen = ImageDataGenerator(rescale=1./255)

        else:
            train_datagen = ImageDataGenerator(rescale=1.0/255)
            validation_datagen = ImageDataGenerator(rescale=1.0/255)

        train_generator = train_datagen.flow_from_directory(
                                            self.train_dir,
                                            classes=['cats', 'dogs'],
                                            target_size=(self.image_size, self.image_size),
                                            batch_size=self.batch_size,
                                            class_mode='binary')
        validation_generator = validation_datagen.flow_from_directory(
                                            self.validation_dir,
                                            classes=['cats', 'dogs'],
                                            target_size=(self.image_size, self.image_size),
                                            batch_size=self.batch_size,
                                            class_mode='binary')

        return train_generator, validation_generator

    def model_building(self):

        ip = Input(batch_shape=(None, self.image_size, self.image_size, 3), name='input')

        x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', name='conv1')(ip)
        x = MaxPool2D(pool_size=(2, 2), name='pool1')(x)

        x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', name='conv2')(x)
        x = MaxPool2D(pool_size=(2, 2), name='pool2')(x)

        x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu', name='conv3')(x)
        x = MaxPool2D(pool_size=(2, 2), name='pool3')(x)

        x = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu', name='conv4')(x)
        x = MaxPool2D(pool_size=(2, 2), name='pool4')(x)

        x = Flatten(name='flatten')(x)
        if self.dropout:
            x = Dropout(rate=0.3, name='dropout1')(x)

        x = Dense(units=128, activation='relu', name='dense1')(x)
        if self.dropout:
            x = Dropout(rate=0.3, name='dropout2')(x)

        output = Dense(units=1, activation='sigmoid', name='output')(x)

        model = Model(inputs=ip, outputs=output)

        return model

    def training_parameters(self):

        if self.optimizer == 'adam':
            opt = optimizers.adam(self.lr)
        elif self.optimizer == 'rms':
            opt = optimizers.RMSprop(self.lr)

        if self.loss == 'binary':
            loss = losses.binary_crossentropy
        elif self.loss == 'multiclass':
            loss = losses.categorical_crossentropy

        if self.metric == 'binary':
            accuracy = metrics.binary_accuracy
        elif self.metric == 'multiclass':
            accuracy = metrics.categorical_crossentropy

        return opt, loss, accuracy

    def training(self):

        self._load_data()

        model = self.model_building()
        model.summary()

        train_generator, validation_generator = self.data_generator()

        opt, loss, accuracy = self.training_parameters()
        model.compile(loss=loss, optimizer=opt, metrics=[accuracy])

        history = model.fit_generator(train_generator,
                                      steps_per_epoch=100,
                                      epochs=self.epochs,
                                      verbose=2,
                                      validation_data=validation_generator,
                                      validation_steps=50)

        model.save(os.path.join(self.LOGDIR + 'cat_dog_v2_1.h5'))

        if self.plotting:

            plt.plot(history.history['loss'], label='training_loss')
            plt.plot(history.history['val_loss'], label='validation_loss')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.title('Training and Validation loss')
            plt.legend()
            plt.show()

            plt.plot(history.history['binary_accuracy'], label='training_accuracy')
            plt.plot(history.history['val_binary_accuracy'], label='validation_accuracy')
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy')
            plt.title('Training and Validation Accuracy')
            plt.legend()
            plt.show()

        return

In [16]:
model = CatsAndDogs(loss='binary', metric='binary',
                    optimizer='adam', lr=1e-4, epochs=20, image_size=150,
                    batch_size=20, augmentation=True, plotting=True,
                    dropout=False, regularizer=False)

In [ ]:
model.training()

Data directory /Volumes/Study/GitHub/data/cats-dogs/
Log directory: /Volumes/Study/GitHub/neural-nets/graphs/cats-dogs/
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 150, 150, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 150, 150, 32)      896       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 75, 75, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 75, 75, 64)        18496     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 37, 37, 64)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 37, 37, 128)       73856     
______________________